In [3]:
import pandas as pd
import os
import json
import csv

In [34]:
# A REMPLIR
consistency_list = []
description_list = []

print(len(consistency_list))
print(len(description_list))

nb_batch = "1"
path_scores_file = "results/" + nb_batch + "/scores3.csv"
df = pd.read_csv(path_scores_file)
df["consistency"] = consistency_list
df["description"] = description_list
df.to_csv('scores.csv', encoding='utf-8')

40
40


### XGEN
#### 4bit

In [36]:

batches = {'0','1'}

for nb_batch in batches:
    name_file = "results/" + nb_batch + "/desc.txt"
    file = open(name_file, 'r', encoding='utf-8')
    description = file.readline()
    print("Batch " + nb_batch + ": " + description)
    # Je souhaite obtenir le dataset des samples cohérents -> filtrage
    path_scores_file = "results/" + nb_batch + "/scores.csv"
    # Extract consistency and description
    df = pd.read_csv(path_scores_file)
    array = df.values

    consistent_df = df[df["consistency"]==1]

    consistent_df_en = consistent_df[consistent_df["output_language"]==1]
    consistent_df_en_en = consistent_df_en[consistent_df_en["input_language"]==1]

    consistent_df_en_fr = consistent_df_en[consistent_df_en["input_language"]==0] # Réponses anglaises à une question française

    consistent_df_fr = consistent_df[consistent_df["output_language"]==0]
    consistent_df_fr_fr = consistent_df_fr[consistent_df_fr["input_language"]==0]

    print("Le batch " + nb_batch + " contient " + str(len(consistent_df)) + "/" + str(len(df)) + " échantillons cohérents.")
    print("Parmi les réponses écrites en français, " + str(len(consistent_df_fr_fr)) + "/" + str(len(consistent_df_fr)) + " des questions sont écrites en français")
    print("Parmi les réponses écrites en anglais, " + str(len(consistent_df_en_en)) + "/" + str(len(consistent_df_en)) + " des questions sont écrites en anglais")
    print()

    rouge2_en_fr = consistent_df_en_fr['rouge2'].mean()
    rougel_en_fr = consistent_df_en_fr['rougel'].mean()
    bert_en_fr = consistent_df_en_fr['bertscore'].mean()

    rouge2_fr_fr = consistent_df_fr_fr['rouge2'].mean()
    rougel_fr_fr = consistent_df_fr_fr['rougel'].mean()
    bert_fr_fr = consistent_df_fr_fr['bertscore'].mean()

    rouge2_en_en = consistent_df_en_en['rouge2'].mean()
    rougel_en_en = consistent_df_en_en['rougel'].mean()
    bert_en_en = consistent_df_en_en['bertscore'].mean()

    print("Moyennes questions françaises, réponses françaises (" + str(len(consistent_df_fr_fr)) + " échantillons):")
    print("Rouge 2: " + str(rouge2_fr_fr) + " ---- Rouge L: " + str(rougel_fr_fr) + " ---- BertScore: " + str(bert_fr_fr))
    print()
    print("Moyennes questions anglaises, réponses anglaises (" + str(len(consistent_df_en_en)) + " échantillons):")
    print("Rouge 2: " + str(rouge2_en_en) + " ---- Rouge L: " + str(rougel_en_en) + " ---- BertScore: " + str(bert_en_en))
    print()
    print("Moyennes questions françaises, réponses anglaises (" + str(len(consistent_df_en_fr)) + " échantillons):")
    print("Rouge 2: " + str(rouge2_en_fr) + " ---- Rouge L: " + str(rougel_en_fr) + " ---- BertScore: " + str(bert_en_fr))
    print()
    print()
    # Grosse tendance à écrire en anglais. Et c'est normal, le header de finetuning est en anglais... Les tâches de finetuning ne sont pas multilingues !
    # Du coup, problème lors du calcul des scores: calculs sur du gold fr et du generated en. De plus, problème plus général sur les métriques, un bertscore français est différent d'un bertscore anglais, pas comparable et la moyenne du rouge score entre deux textes randoms en langue anglais et française est probablement différente... Limites de ces mesures. Justifie l'analyse qualitative.


Batch 0: This inference batch is aimed at testing this code. It helped debugging. it was performed with a 4bit quantized version of XGen 7B. One take away is that for 4 out of the 5 fredsum texts, there was too many tokens (above 1800)
Le batch 0 contient 28/40 échantillons cohérents.
Parmi les réponses écrites en français, 1/1 des questions sont écrites en français
Parmi les réponses écrites en anglais, 16/27 des questions sont écrites en anglais

Moyennes questions françaises, réponses françaises (1 échantillons):
Rouge 2: 0.0574162679425837 ---- Rouge L: 0.0904761904761904 ---- BertScore: -0.0642949044704437

Moyennes questions anglaises, réponses anglaises (16 échantillons):
Rouge 2: 0.10337345389955058 ---- Rouge L: 0.19059100544868074 ---- BertScore: 0.2400263991439715

Moyennes questions françaises, réponses anglaises (11 échantillons):
Rouge 2: 0.06339291110835027 ---- Rouge L: 0.13676942533994166 ---- BertScore: 0.16332498328252268


Batch 1: This inference batch is aimed at t